In [1]:
import tensorflow

In [2]:
import keras

Using TensorFlow backend.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt 
import datetime
import numpy as np

In [4]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE

In [5]:
train_df = pd.read_csv("train.csv")

In [6]:
train_df.columns

Index(['id', 'amount_value', 'amount_currency', 'channel',
       'deviceDetails_browser', 'deviceDetails_device',
       'deviceDetails_deviceIp', 'merchantRefTransactionId',
       'paymentMethod_apmType', 'paymentMethod_cardNumber',
       'paymentMethod_cardType', 'paymentMethod_cardSubType',
       'paymentMethod_cvv', 'paymentMethod_encodedPaymentToken',
       'paymentMethod_expiryMonth', 'paymentMethod_expiryYear',
       'transaction_time', 'class'],
      dtype='object')

In [7]:
train_df.head()

,id,amount_value,amount_currency,channel,deviceDetails_browser,deviceDetails_device,deviceDetails_deviceIp,merchantRefTransactionId,paymentMethod_apmType,paymentMethod_cardNumber,paymentMethod_cardType,paymentMethod_cardSubType,paymentMethod_cvv,paymentMethod_encodedPaymentToken,paymentMethod_expiryMonth,paymentMethod_expiryYear,transaction_time,class
0,0,80073.39,USD,pos,chrome,pos,199.177.94.32,9196816658,magstripe,8717-6498-5500-4384,JCB,Student,802,RGNTuKCD,4,2023,2020-08-06,1
1,1,44730.51,USD,virtual,mozilla,pos,140.130.6.118,2480479816,magstripe,5910-2388-3992-2019,Discover,Prepaid,290,ECEPBpho,8,2021,2020-09-08,0
2,2,6068.00,USD,virtual,chrome,pos,123.65.222.211,7316485036,nfcc,7491-3848-0599-6446,JCB,Student,298,sKBUDqEt,9,2020,2020-01-01,1
3,3,93995.29,USD,pos,chrome,pos,53.155.4.150,666192529,nfcc,8882-7459-8781-3143,MasterCard,Student,915,dLwxtjDg,10,2023,2020-02-07,1
4,4,28724.03,USD,online,edge,pos,169.106.202.210,3311701970,magstripe,5234-6985-8914-2457,JCB,Student,771,nlxiTCcp,11,2024,2020-01-24,1


In [8]:
#transform transaction time in day of the week
# 0 is monday
def get_day(x):
    today = datetime.datetime.strptime(x, "%Y-%m-%d")
    return today.weekday()

In [9]:
train_df['transaction_time'] = train_df['transaction_time'].apply(get_day)
train_df['transaction_time']

0        3
1        1
2        2
3        4
4        4
        ..
99995    4
99996    2
99997    2
99998    3
99999    6
Name: transaction_time, Length: 100000, dtype: int64

In [10]:
class_df = train_df['class'].value_counts().to_frame().reset_index()
class_df['percentages'] = class_df['class'].apply(lambda val: round(100*float(val)/len(train_df), 2))
class_df

,index,class,percentages
0,1,50084,50.08
1,0,49916,49.92


In [48]:
non_fraud = train_df[train_df['class'] == 0].sample(1000)
fraud = train_df[train_df['class'] == 1].sample(1000)

df = non_fraud.append(fraud).reset_index(drop=True)
X = df.drop(['class'], axis=1)
X = X[['amount_value', 'merchantRefTransactionId', 'paymentMethod_cvv', 'paymentMethod_expiryMonth', 'paymentMethod_expiryYear', 'transaction_time']]
X = X.values

Y = df['class'].values
X

array([[1.36123600e+04, 3.40280360e+07, 6.54000000e+02, 1.00000000e+01,
        2.02700000e+03, 3.00000000e+00],
       [2.34430500e+04, 5.75522402e+09, 1.61000000e+02, 9.00000000e+00,
        2.01900000e+03, 2.00000000e+00],
       [6.36077000e+03, 2.50702960e+09, 8.91000000e+02, 1.00000000e+00,
        2.02200000e+03, 6.00000000e+00],
       ...,
       [2.11581600e+04, 7.91676413e+09, 2.82000000e+02, 7.00000000e+00,
        2.02000000e+03, 2.00000000e+00],
       [1.52977000e+04, 1.37000513e+09, 1.77000000e+02, 8.00000000e+00,
        2.02300000e+03, 0.00000000e+00],
       [6.70704000e+03, 2.16150361e+09, 4.87000000e+02, 3.00000000e+00,
        2.02000000e+03, 0.00000000e+00]])

In [46]:
def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1) # fits the data into a 2D embedded space

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label='Non Fraud')
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label='Fraud')

    plt.legend(loc='best');
    plt.savefig(name)
    plt.show();
    

In [47]:
tsne_plot(X, Y, "original.png")

TypeError: must be real number, not str

<Figure size 864x576 with 1 Axes>